In [21]:
import collections
import json
import re
from itertools import chain
from multiprocessing import Pool
from pathlib import Path
from typing import *

import dotenv
import git
import pandas as pd
import pydriller
from pymongo import MongoClient

In [22]:
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])
DATA_DIR, DATA_DIR.exists()

(PosixPath('/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data'), True)

In [23]:
client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio
patch_urls_mongo_collection = db.patchUrls
df = pd.DataFrame(list(patch_urls_mongo_collection.find()))
df

,_id,Id@,VulnUrl@,PatchUrls@,VulnUrl,PatchUrls
0,6556d453917e89644f58feb9,6556d453917e89644f58feb9,https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[],https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[]
1,6556d453917e89644f58feba,6556d453917e89644f58feba,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/626c...,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/626c...
2,6556d453917e89644f58febb,6556d453917e89644f58febb,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...
3,6556d453917e89644f58febc,6556d453917e89644f58febc,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...
4,6556d453917e89644f58febd,6556d453917e89644f58febd,https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[],https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[]
...,...,...,...,...,...,...
673,6556d4ac917e89644f59015a,6556d4ac917e89644f59015a,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[],https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[]
674,6556d4ac917e89644f59015b,6556d4ac917e89644f59015b,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/99...,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/99...
675,6556d4ac917e89644f59015c,6556d4ac917e89644f59015c,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...
676,6556d4ac917e89644f59015d,6556d4ac917e89644f59015d,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[],https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[]


In [24]:
df = df[df['PatchUrls'].map(len) > 0]
df

,_id,Id@,VulnUrl@,PatchUrls@,VulnUrl,PatchUrls
1,6556d453917e89644f58feba,6556d453917e89644f58feba,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/626c...,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/626c...
2,6556d453917e89644f58febb,6556d453917e89644f58febb,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...
3,6556d453917e89644f58febc,6556d453917e89644f58febc,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...,https://security.snyk.io/vuln/SNYK-JAVA-CHQOSL...,[https://github.com/qos-ch/logback/commit/f460...
5,6556d453917e89644f58febe,6556d453917e89644f58febe,https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[https://github.com/dromara/hutool/commit/c335...,https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[https://github.com/dromara/hutool/commit/c335...
6,6556d453917e89644f58febf,6556d453917e89644f58febf,https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[https://github.com/dromara/hutool/commit/9ba8...,https://security.snyk.io/vuln/SNYK-JAVA-CNHUTO...,[https://github.com/dromara/hutool/commit/9ba8...
...,...,...,...,...,...,...
668,6556d4ab917e89644f590155,6556d4ab917e89644f590155,https://security.snyk.io/vuln/SNYK-JAVA-ORGXER...,[https://github.com/xerial/snappy-java/commit/...,https://security.snyk.io/vuln/SNYK-JAVA-ORGXER...,[https://github.com/xerial/snappy-java/commit/...
670,6556d4ab917e89644f590157,6556d4ab917e89644f590157,https://security.snyk.io/vuln/SNYK-JAVA-ORGZER...,[https://github.com/zeroturnaround/zt-zip/comm...,https://security.snyk.io/vuln/SNYK-JAVA-ORGZER...,[https://github.com/zeroturnaround/zt-zip/comm...
674,6556d4ac917e89644f59015b,6556d4ac917e89644f59015b,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/99...,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/99...
675,6556d4ac917e89644f59015c,6556d4ac917e89644f59015c,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...,https://security.snyk.io/vuln/SNYK-JAVA-XERCES...,[https://github.com/apache/xerces2-j/commit/0b...


In [25]:
df_dict = df.to_dict(orient='records')
df_dict

[{'_id': ObjectId('6556d453917e89644f58feba'),
  'Id@': ObjectId('6556d453917e89644f58feba'),
  'VulnUrl@': 'https://security.snyk.io/vuln/SNYK-JAVA-CHQOSLOGBACK-1726923',
  'PatchUrls@': ['https://github.com/qos-ch/logback/commit/626c7733c188f2ad60c1348a493761f60e2d7958'],
  'VulnUrl': 'https://security.snyk.io/vuln/SNYK-JAVA-CHQOSLOGBACK-1726923',
  'PatchUrls': ['https://github.com/qos-ch/logback/commit/626c7733c188f2ad60c1348a493761f60e2d7958']},
 {'_id': ObjectId('6556d453917e89644f58febb'),
  'Id@': ObjectId('6556d453917e89644f58febb'),
  'VulnUrl@': 'https://security.snyk.io/vuln/SNYK-JAVA-CHQOSLOGBACK-31407',
  'PatchUrls@': ['https://github.com/qos-ch/logback/commit/f46044b805bca91efe5fd6afe52257cd02f775f8',
   'https://github.com/qos-ch/logback/commit/979b042cb1f0b4c1e5869ccc8912e68c39f769f9'],
  'VulnUrl': 'https://security.snyk.io/vuln/SNYK-JAVA-CHQOSLOGBACK-31407',
  'PatchUrls': ['https://github.com/qos-ch/logback/commit/f46044b805bca91efe5fd6afe52257cd02f775f8',
   'http

In [26]:
# patch_url -> vuln_url
patch_urls = collections.defaultdict(set)
for r in df_dict:
    for pu in r['PatchUrls']:
        patch_urls[pu].add(r['VulnUrl'])
len(patch_urls.keys())

484

In [27]:
def get_repo_and_commit(commit_url: str) -> Tuple[str, str]:
    for m in re.finditer(r"(?P<repo>(https://)?(www\.)?github\.com(?:/[^/]+)*)/commit/(?P<hash>[0-9a-f]+)(\#diff.*)?",
                         commit_url):
        if(m.group('repo') == "https://github.com/looly/hutool"):
            return "https://github.com/dromara/hutool", m.group('hash')
        return m.group('repo'), m.group('hash')

In [28]:
repo_patches = collections.defaultdict(set)
repo_commit_to_url = dict()
for p in patch_urls.keys():
    print(p)
    repo, commit = get_repo_and_commit(p)
    repo_patches[repo].add(commit)
    repo_commit_to_url[(repo, commit)] = p

repo_patches, len(repo_patches.keys()), len(list(chain(*repo_patches.values())))

https://github.com/qos-ch/logback/commit/626c7733c188f2ad60c1348a493761f60e2d7958
https://github.com/qos-ch/logback/commit/f46044b805bca91efe5fd6afe52257cd02f775f8
https://github.com/qos-ch/logback/commit/979b042cb1f0b4c1e5869ccc8912e68c39f769f9
https://github.com/dromara/hutool/commit/c33550f703f5d1d7dd71ad2992d79a5e5532ce2c
https://github.com/dromara/hutool/commit/9ba8f9ca5dd32441f2e0f150cb22fa178bb771d3
https://github.com/dromara/hutool/commit/2a786afd3fef5554321d919481aeac8ed362ce99
https://github.com/dromara/hutool/commit/5c4486b9f58a83f283868135138f6ff3741b8c12
https://github.com/looly/hutool/commit/8d7d0b7fb5ea4f7447b40131bffc1ec506a6528e
https://github.com/looly/hutool/commit/9f8a801c7b98b75ee681c0988e1a58bcfdc21756
https://github.com/looly/hutool/commit/fed1a1f747a9308e2f65f8dbbff05ce62478ecc0
https://github.com/alibaba/fastjson/commit/35db4adad70c32089542f23c272def1ad920a60d
https://github.com/alibaba/fastjson/commit/8f3410f81cbd437f7c459f8868445d50ad301f15
https://github.com

(defaultdict(set,
             {'https://github.com/qos-ch/logback': {'626c7733c188f2ad60c1348a493761f60e2d7958',
               '979b042cb1f0b4c1e5869ccc8912e68c39f769f9',
               'f46044b805bca91efe5fd6afe52257cd02f775f8'},
              'https://github.com/dromara/hutool': {'2a786afd3fef5554321d919481aeac8ed362ce99',
               '5c4486b9f58a83f283868135138f6ff3741b8c12',
               '8d7d0b7fb5ea4f7447b40131bffc1ec506a6528e',
               '9ba8f9ca5dd32441f2e0f150cb22fa178bb771d3',
               '9f8a801c7b98b75ee681c0988e1a58bcfdc21756',
               'c33550f703f5d1d7dd71ad2992d79a5e5532ce2c',
               'fed1a1f747a9308e2f65f8dbbff05ce62478ecc0'},
              'https://github.com/alibaba/fastjson': {'35db4adad70c32089542f23c272def1ad920a60d',
               '8f3410f81cbd437f7c459f8868445d50ad301f15'},
              'https://github.com/aerospike/aerospike-client-java': {'02bf28e62fb186f004c82c87b219db2fc5b8262a',
               '51c65e32837da29435161a2d9c09b

In [ ]:
repo_path = DATA_DIR / 'interim' / 'repositories' /'temp'
repo_path.mkdir(exist_ok=True, parents=True)

def get_new_records(repo_url) -> Tuple[List, Dict]:
    error_data = {
        'clone': [],
        'git': [],
        'no_changed_method_gavs': [],
        'no_java_gavs': [],
        'no_source_code': [],
        'is_test': [],
        'traversal_problems': [],
        'merge_commits': [],
        'no_modifications': [],
    }
    new_records = list()
    commit_hashes = repo_patches[repo_url]

    def original_commit_hash(commit_hash: str) -> str:
        return next(x for x in commit_hashes if (x.startswith(commit_hash) or commit_hash.startswith(x)))

    try:
        repo = pydriller.Repository(repo_url, clone_repo_to=repo_path, include_remotes=True, include_deleted_files=True,
                                    include_refs=True)
    except Exception:
        print(f'error cloning {repo_url}')
        error_data['clone'].append(repo_url)
        return [], error_data

    try:
        repo_commits = [c for c in repo.traverse_commits()]
    except git.exc.CommandError as e:
        print(f'git cmd error ({repo_url}): [{type(e)}: {e}]')
        error_data['git'].append(repo_url)
        return [], error_data
    except Exception as e:
        print(f'error traversing commits ({repo_url}): [{type(e)}: {e}]')
        error_data['git'].append(repo_url)
        return [], error_data

    for ch in commit_hashes:
        to_be_traversed = list(c for c in repo_commits if c.hash.startswith(ch) or ch.startswith(c.hash))
        if len(to_be_traversed) != 1:
            error_data['traversal_problems'].append((repo_url, ch))
            print(f'there is a traversal problem for {(repo_url, ch, len(to_be_traversed))}')

        for commit in to_be_traversed:
            if len(commit.modified_files) == 0:
                if commit.merge:
                    print(f'merge commit {(repo_url, commit.hash)}')
                    error_data['merge_commits'].append((repo_url, commit.hash))
                else:
                    print(f'commit has no modified files: {(repo_url, commit.hash)}')
                    error_data['no_modifications'].append((repo_url, commit.hash))

                continue

            modified_java_files = list(mf for mf in commit.modified_files if mf.filename.endswith('.java'))
            if len(modified_java_files) == 0:
                print(f'no_java_gavs {(repo_url, commit.hash)}')
                error_data['no_java_gavs'].append((repo_url, commit.hash))
                continue

            modified_java_files = list(mf for mf in modified_java_files if mf.source_code_before is not None)
            if len(modified_java_files) == 0:
                print(f'no_source_code {(repo_url, commit.hash)}')
                error_data['no_source_code'].append((repo_url, commit.hash))
                continue
            modified_java_files = list(mf for mf in modified_java_files if len(mf.changed_methods) > 0)
            if len(modified_java_files) == 0:
                print(f'no_changed_method_gavs {(repo_url, commit.hash)}')
                error_data['no_changed_method_gavs'].append((repo_url, commit.hash))
                continue

            modified_java_files = list(
                mf for mf in modified_java_files if len(re.compile(r'[Tt]est').findall(mf.old_path)) == 0)
            if len(modified_java_files) == 0:
                print(f'is_test {(repo_url, commit.hash)}')
                error_data['is_test'].append((repo_url, commit.hash))
                continue



            for mf in modified_java_files:
                try:
                    new_row = dict()
                    new_row['repo'] = repo_url
                    new_row['commitHash'] = ch
                    new_row['snykPatchUrl'] = repo_commit_to_url[(repo_url, original_commit_hash(commit.hash))]
                    new_row['commitHash'] = commit.hash
                    new_row['modifiedFilePathBefore'] = mf.old_path
                    new_row['modifiedFilePathAfter'] = mf.new_path
                    new_row['modifiedFileSrcBefore'] = mf.source_code_before
                    new_row['modifiedFileSrcAfter'] = mf.source_code
                    new_row['diffParsedJson'] = json.dumps(mf.diff_parsed)
                    new_row['nloc'] = mf.nloc
                    new_row['changedMethods'] = list(map(lambda m: m.name, mf.changed_methods))

                    new_records.append(new_row)

                except Exception:
                    print(f'error parsing modified files {repo_url}: {ch} ({mf.filename})')
                    continue
    return new_records, error_data

In [30]:
#res = []
#for key in repo_patches.keys():
#    res.append(get_new_records(key))
    

with Pool(16) as p:
   res = p.map(get_new_records, repo_patches.keys())

no_java_gavs ('https://github.com/jmrozanec/cron-utils', '4cf373f7352f5d95f0bf6512af8af326b31c835e')
merge commit ('https://github.com/jmrozanec/cron-utils', 'd6707503ec2f20947f79e38f861dba93b39df9da')
no_java_gavs ('https://github.com/OWASP/java-html-sanitizer', 'e2b29e803c4f7ea225f569d18b3b1356d8e5bf3d')
no_java_gavs ('https://github.com/cbeust/jcommander', '3ae95595febbed9c13f367b6bda5c0be1c572c53')
no_java_gavs ('https://github.com/itext/itextsharp', '72a8d567ce0b20c549669258145f8d4b19e0fa4e')
merge commit ('https://github.com/FasterXML/jackson-modules-java8', 'f29e03e21eb6ca0c11cda34cf511d245711a70ec')
no_java_gavs ('https://github.com/itext/itext7-dotnet', '976884e47a265fb2114756ffacc04b913f9c4dcb')
is_test ('https://github.com/qos-ch/logback', 'f46044b805bca91efe5fd6afe52257cd02f775f8')
there is a traversal problem for ('https://github.com/graphql-java/graphql-java', '8b9617f55876505dd1ac547436a3fd9de60f2d55', 0)
no_changed_method_gavs ('https://github.com/graphql-java/graphql-j

In [31]:
new_df = list(chain(*[x[0] for x in res]))
len(new_df), new_df[0].keys()

(1153,
 dict_keys(['repo', 'commitHash', 'snykPatchUrl', 'modifiedFilePathBefore', 'modifiedFilePathAfter', 'modifiedFileSrcBefore', 'modifiedFileSrcAfter', 'diffParsedJson', 'nloc', 'changedMethods']))

In [32]:
errors = collections.defaultdict(set)
for e in [r[1] for r in res]:
    for k, v in e.items():
        errors[k].update(v)
for k, v in errors.items():
    print(f'{k}: {len(v)}')
errors

clone: 0
git: 0
no_changed_method_gavs: 33
no_java_gavs: 20
no_source_code: 1
is_test: 14
traversal_problems: 7
merge_commits: 20
no_modifications: 0


defaultdict(set,
            {'clone': set(),
             'git': set(),
             'no_changed_method_gavs': {('https://github.com/FasterXML/jackson-databind',
               '05d7e0e13f43e12db6a51726df12c8b4d8040676'),
              ('https://github.com/FasterXML/jackson-databind',
               '1645efbd392989cf015f459a91c999e59c921b15'),
              ('https://github.com/FasterXML/jackson-databind',
               '191a4cdf87b56d2ddddb77edd895ee756b7f75eb'),
              ('https://github.com/FasterXML/jackson-databind',
               '27b4defc270454dea6842bd9279f17387eceb737'),
              ('https://github.com/FasterXML/jackson-databind',
               '28badf7ef60ac3e7ef151cd8e8ec010b8479226a'),
              ('https://github.com/FasterXML/jackson-databind',
               '3e8fa3beea49ea62109df9e643c9cb678dabdde1'),
              ('https://github.com/FasterXML/jackson-databind',
               '41b7f9b90149e9d44a65a8261a8deedc7186f6af'),
              ('https://github.co

In [33]:
commit_urls = set()
commits = set()
repos = set()
for it in new_df:
    commit_urls.add((it['repo'], it['commitHash']))
    repos.add(it['repo'])
    commits.add(it['commitHash'])
len(repos), len(commit_urls)  # 87, 266 commits

(95, 385)

In [34]:
res_df = pd.DataFrame(data=new_df)
res_df  # 1050 files

,repo,commitHash,snykPatchUrl,modifiedFilePathBefore,modifiedFilePathAfter,modifiedFileSrcBefore,modifiedFileSrcAfter,diffParsedJson,nloc,changedMethods
0,https://github.com/qos-ch/logback,626c7733c188f2ad60c1348a493761f60e2d7958,https://github.com/qos-ch/logback/commit/626c7...,logback-core/src/main/java/ch/qos/logback/core...,logback-core/src/main/java/ch/qos/logback/core...,"/**\n * Logback: the reliable, generic, fast a...","/**\n * Logback: the reliable, generic, fast a...","{""added"": [[194, "" } catch (jav...",211.0,[AbstractSocketAppender::connectSocketAndDispa...
1,https://github.com/qos-ch/logback,626c7733c188f2ad60c1348a493761f60e2d7958,https://github.com/qos-ch/logback/commit/626c7...,logback-core/src/main/java/ch/qos/logback/core...,logback-core/src/main/java/ch/qos/logback/core...,"/**\n * Logback: the reliable, generic, fast a...","/**\n * Logback: the reliable, generic, fast a...","{""added"": [[22, "" * @author Bruno Harbulot""], ...",35.0,[SSLConfigurableServerSocket::setHostnameVerif...
2,https://github.com/qos-ch/logback,626c7733c188f2ad60c1348a493761f60e2d7958,https://github.com/qos-ch/logback/commit/626c7...,logback-core/src/main/java/ch/qos/logback/core...,logback-core/src/main/java/ch/qos/logback/core...,"/**\n * Logback: the reliable, generic, fast a...","/**\n * Logback: the reliable, generic, fast a...","{""added"": [[16, ""import javax.net.ssl.SSLParam...",42.0,[SSLConfigurableSocket::setHostnameVerification]
3,https://github.com/qos-ch/logback,626c7733c188f2ad60c1348a493761f60e2d7958,https://github.com/qos-ch/logback/commit/626c7...,logback-core/src/main/java/ch/qos/logback/core...,logback-core/src/main/java/ch/qos/logback/core...,"/**\n * Logback: the reliable, generic, fast a...","/**\n * Logback: the reliable, generic, fast a...","{""added"": [[32, "" * @author Bruno Harbulot""], ...",118.0,[SSLParametersConfiguration::setHostnameVerifi...
4,https://github.com/qos-ch/logback,979b042cb1f0b4c1e5869ccc8912e68c39f769f9,https://github.com/qos-ch/logback/commit/979b0...,logback-access/src/main/java/ch/qos/logback/ac...,logback-access/src/main/java/ch/qos/logback/ac...,"/**\n * Logback: the reliable, generic, fast a...","/**\n * Logback: the reliable, generic, fast a...","{""added"": [[44, "" HardenedAccessEventInputS...",48.0,"[SocketNode::run, SocketNode::SocketNode]"
...,...,...,...,...,...,...,...,...,...,...
1148,https://github.com/apache/xerces2-j,992b5d9c24102ad20330d36c0a71162753a37449,https://github.com/apache/xerces2-j/commit/992...,src/org/apache/xerces/util/SoftReferenceSymbol...,src/org/apache/xerces/util/SoftReferenceSymbol...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"{""added"": [[130, "" int collisionCount =...",236.0,"[SoftReferenceSymbolTable::rebalance, SoftRefe..."
1149,https://github.com/apache/xerces2-j,992b5d9c24102ad20330d36c0a71162753a37449,https://github.com/apache/xerces2-j/commit/992...,src/org/apache/xerces/util/SymbolHash.java,src/org/apache/xerces/util/SymbolHash.java,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"{""added"": [[34, """"], [36, "" protected stati...",161.0,"[SymbolHash::rebalance, SymbolHash::rehashComm..."
1150,https://github.com/apache/xerces2-j,992b5d9c24102ad20330d36c0a71162753a37449,https://github.com/apache/xerces2-j/commit/992...,src/org/apache/xerces/util/SymbolTable.java,src/org/apache/xerces/util/SymbolTable.java,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"{""added"": [[87, """"], [88, "" /** Maximum has...",193.0,"[SymbolTable::rebalance, SymbolTable::addSymbo..."
1151,https://github.com/apache/xerces2-j,992b5d9c24102ad20330d36c0a71162753a37449,https://github.com/apache/xerces2-j/commit/992...,src/org/apache/xerces/util/XMLAttributesImpl.java,src/org/apache/xerces/util/XMLAttributesImpl.java,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Softwar

In [35]:
db.patchCommitsLibio.drop()
db.patchCommitsLibio.insert_many(new_df)

InsertManyResult([ObjectId('67fc39878f286857456a8ebc'), ObjectId('67fc39878f286857456a8ebd'), ObjectId('67fc39878f286857456a8ebe'), ObjectId('67fc39878f286857456a8ebf'), ObjectId('67fc39878f286857456a8ec0'), ObjectId('67fc39878f286857456a8ec1'), ObjectId('67fc39878f286857456a8ec2'), ObjectId('67fc39878f286857456a8ec3'), ObjectId('67fc39878f286857456a8ec4'), ObjectId('67fc39878f286857456a8ec5'), ObjectId('67fc39878f286857456a8ec6'), ObjectId('67fc39878f286857456a8ec7'), ObjectId('67fc39878f286857456a8ec8'), ObjectId('67fc39878f286857456a8ec9'), ObjectId('67fc39878f286857456a8eca'), ObjectId('67fc39878f286857456a8ecb'), ObjectId('67fc39878f286857456a8ecc'), ObjectId('67fc39878f286857456a8ecd'), ObjectId('67fc39878f286857456a8ece'), ObjectId('67fc39878f286857456a8ecf'), ObjectId('67fc39878f286857456a8ed0'), ObjectId('67fc39878f286857456a8ed1'), ObjectId('67fc39878f286857456a8ed2'), ObjectId('67fc39878f286857456a8ed3'), ObjectId('67fc39878f286857456a8ed4'), ObjectId('67fc39878f286857456a8e